In [ ]:
import json
import requests
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField, 
    IntegerType, StringType, FloatType, DoubleType,
    DateType, TimestampType, BooleanType, DecimalType
)

In [ ]:
table_name = ""
schema_json = ""
drop_before_load = "true"

In [ ]:
print("=" * 70)
print("FULL LOAD TABLE ")
print("=" * 70)
print(f"\nParameters nhận được:")
print(f"  ├─ table_name: {table_name if table_name else '(empty)'}")
print(f"  └─ drop_before_load: {drop_before_load}")

# Validate
if not table_name:
    raise ValueError("Thiếu parameter 'table_name'!")

if not schema_json:
    raise ValueError("Thiếu parameter 'schema_json'!")

# Convert drop_before_load to boolean
drop_before_load_bool = drop_before_load.lower() == "true" if isinstance(drop_before_load, str) else drop_before_load


In [ ]:
def get_supabase_credentials():
    """Load Supabase credentials từ Lakehouse"""
    config_path = "/lakehouse/default/Files/.config/credentials.json"
    
    try:
        with open(config_path, "r") as f:
            creds = json.load(f)
        return creds["supabase"]["url"], creds["supabase"]["key"]
    except FileNotFoundError:
        print("Chưa setup credentials!")
        return None, None
    except Exception as e:
        print(f"Lỗi đọc config: {e}")
        return None, None


In [ ]:
def parse_schema_from_json(schema_json_str: str) -> StructType:
    """Parse schema từ JSON string"""
    try:
        schema_dict = json.loads(schema_json_str)
        
        type_mapping = {
            "IntegerType": IntegerType(),
            "StringType": StringType(),
            "FloatType": FloatType(),
            "DoubleType": DoubleType(),
            "DateType": DateType(),
            "TimestampType": TimestampType(),
            "BooleanType": BooleanType(),
            "DecimalType": lambda p, s: DecimalType(p, s)
        }
        
        fields = []
        for field_dict in schema_dict["fields"]:
            field_name = field_dict["name"]
            field_type_str = field_dict["type"]
            nullable = field_dict.get("nullable", True)
            
            # Xử lý DecimalType
            if field_type_str.startswith("DecimalType"):
                import re
                match = re.search(r'DecimalType\((\d+),\s*(\d+)\)', field_type_str)
                if match:
                    precision, scale = int(match.group(1)), int(match.group(2))
                    field_type = DecimalType(precision, scale)
                else:
                    field_type = DecimalType(19, 4)  # Default
            else:
                field_type = type_mapping.get(field_type_str, StringType())
            
            fields.append(StructField(field_name, field_type, nullable))
        
        return StructType(fields)
        
    except Exception as e:
        print(f"Lỗi parse schema: {e}")
        raise

In [ ]:
def fetch_table_data(base_url: str, api_key: str, table_name: str, 
                     batch_size: int = 1000) -> pd.DataFrame:
    """Lấy dữ liệu từ 1 bảng Supabase qua REST API"""
    headers = {
        "apikey": api_key,
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    all_data = []
    offset = 0
    
    print(f"\nĐang lấy dữ liệu từ bảng: {table_name}")
    
    while True:
        url = f"{base_url}/rest/v1/{table_name}"
        
        params = {
            "select": "*",
            "limit": batch_size,
            "offset": offset
        }
        
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            
            data = response.json()
            
            if not data:
                break
            
            all_data.extend(data)
            print(f"  ├─ Đã lấy {len(all_data)} records...")
            
            if len(data) < batch_size:
                break
            
            offset += batch_size
            
        except requests.exceptions.RequestException as e:
            print(f"Lỗi request: {e}")
            return pd.DataFrame()
    
    if all_data:
        df = pd.DataFrame(all_data)
        print(f"Hoàn tất: {len(df)} records x {len(df.columns)} columns")
        return df
    else:
        print(f"Không có dữ liệu")
        return pd.DataFrame()

In [ ]:
def create_watermark_table_if_not_exists(spark):
    """Tạo bảng watermark nếu chưa tồn tại"""
    try:
        watermark_table = "water_mark"
        
        # Kiểm tra bảng có tồn tại không
        tables = [t.name for t in spark.catalog.listTables()]
        
        if watermark_table not in tables:
            print(f"\nTạo bảng watermark...")
            
            create_query = f"""
            CREATE TABLE IF NOT EXISTS {watermark_table} (
                table_name STRING,
                max_modified_date TIMESTAMP,
                layer_type STRING,
                last_load_time TIMESTAMP,
                row_count BIGINT
            ) USING DELTA
            """
            spark.sql(create_query)
            print(f"Đã tạo bảng {watermark_table}")
            
    except Exception as e:
        print(f"Lỗi khi tạo bảng watermark: {e}")
        raise

In [ ]:
def update_watermark(spark, table_name: str, pandas_df: pd.DataFrame, 
                     layer_type: str = "bronze"):
    """Cập nhật watermark cho 1 bảng"""
    try:
        create_watermark_table_if_not_exists(spark)
        
        watermark_table = "water_mark"
        
        # Tìm max_modified_date
        if 'ModifiedDate' in pandas_df.columns:
            pandas_df['ModifiedDate'] = pd.to_datetime(pandas_df['ModifiedDate'])
            max_modified = pandas_df['ModifiedDate'].max()
            
            if pd.notna(max_modified):
                max_modified = max_modified.to_pydatetime()
            else:
                max_modified = datetime.now()
                print(f"ModifiedDate có giá trị NULL, dùng datetime.now()")
        else:
            print(f"Không có cột ModifiedDate, dùng datetime.now()")
            max_modified = datetime.now()
        
        # Tạo record watermark
        watermark_record = [{
            'table_name': table_name,
            'max_modified_date': max_modified,
            'layer_type': layer_type,
            'last_load_time': datetime.now(),
            'row_count': len(pandas_df)
        }]
        
        watermark_df = spark.createDataFrame(watermark_record)
        watermark_df.createOrReplaceTempView("temp_watermark")
        
        merge_query = f"""
        MERGE INTO {watermark_table} AS target
        USING temp_watermark AS source
        ON target.table_name = source.table_name
        WHEN MATCHED THEN
            UPDATE SET 
                max_modified_date = source.max_modified_date,
                last_load_time = source.last_load_time,
                layer_type = source.layer_type,
                row_count = source.row_count
        WHEN NOT MATCHED THEN
            INSERT (table_name, max_modified_date, layer_type, last_load_time, row_count)
            VALUES (source.table_name, source.max_modified_date, source.layer_type, 
                    source.last_load_time, source.row_count)
        """
        
        spark.sql(merge_query)
        
        print(f"\nĐã cập nhật watermark:")
        print(f"  ├─ Table: {table_name}")
        print(f"  ├─ Max Modified Date: {max_modified}")
        print(f"  ├─ Row Count: {len(pandas_df):,}")
        print(f"  └─ Load Time: {datetime.now()}")
        
    except Exception as e:
        print(f"\nLỗi khi cập nhật watermark: {e}")
        raise

In [ ]:
print("\n" + "=" * 70)
print(f"LOAD TABLE: {table_name}")
print("=" * 70)

try:
    # Bước 1: Parse schema
    print(f"\nParse schema...")
    schema = parse_schema_from_json(schema_json)
    print(f"Schema có {len(schema.fields)} cột")
    
    # Bước 2: Lấy dữ liệu từ Supabase
    url, key = get_supabase_credentials()
    if not url or not key:
        raise ValueError("Không thể lấy credentials!")
    
    pandas_df = fetch_table_data(url, key, table_name)
    
    if pandas_df.empty:
        raise ValueError("Không có dữ liệu để load!")
    
    # Bước 3: Khởi tạo Spark
    spark = SparkSession.builder.appName(f"Load_{table_name}").getOrCreate()
    
    # Bước 4: Convert sang Spark DataFrame với schema
    print(f"\nChuyển đổi DataFrame với schema...")
    try:
        # # Convert các cột datetime trong pandas trước
        # for field in schema.fields:
        #     if isinstance(field.dataType, TimestampType) and field.name in pandas_df.columns:
        #         pandas_df[field.name] = pd.to_datetime(pandas_df[field.name], errors='coerce')
        print(f"  ├─ Đang convert timestamp columns...")
        for field in schema.fields:
            if field.name in pandas_df.columns:
                if isinstance(field.dataType, TimestampType):
                    print(f"    ├─ {field.name}: String → Timestamp")
                    pandas_df[field.name] = pd.to_datetime(pandas_df[field.name], errors='coerce', utc=True)
                    # Remove timezone để tránh lỗi với Spark
                    pandas_df[field.name] = pandas_df[field.name].dt.tz_localize(None)
                elif isinstance(field.dataType, DateType):
                    print(f"    ├─ {field.name}: String → Date")
                    pandas_df[field.name] = pd.to_datetime(pandas_df[field.name], errors='coerce').dt.date
        
        # Show sample data sau khi convert
        print(f"\n  📋 Sample data sau convert:")
        print(f"    {pandas_df.dtypes.to_dict()}")
        
        spark_df = spark.createDataFrame(pandas_df, schema=schema)
        print(f"Đã áp dụng schema")
        
        # Hiển thị schema
        print(f"\nSchema của bảng:")
        spark_df.printSchema()
        
    except Exception as e:
        print(f"Lỗi khi áp dụng schema: {e}")
        print("Đang thử load mà không ép schema...")
        spark_df = spark.createDataFrame(pandas_df)
    
    # Bước 5: Ghi vào Lakehouse (không dùng schema prefix)
    print(f"\nGhi dữ liệu vào: {table_name}")
    
    # DROP TABLE nếu cần
    if drop_before_load_bool and spark.catalog.tableExists(table_name):
        print(f"  ├─ DROP TABLE {table_name}")
        spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    
    # Ghi vào bảng Delta
    spark_df.write \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .format("delta") \
        .saveAsTable(table_name)
    
    row_count = spark_df.count()
    print(f"Đã ghi {row_count:,} rows vào {table_name}")
    
    # Bước 6: Cập nhật watermark
    print(f"\nCập nhật watermark...")
    update_watermark(spark, table_name, pandas_df)
    
    # Bước 7: Hiển thị kết quả
    print("\n" + "=" * 70)
    print("HOÀN TẤT LOAD TABLE")
    print("=" * 70)
    
    print(f"\nThống kê:")
    print(f"  ├─ Table: {table_name}")
    print(f"  ├─ Rows: {row_count:,}")
    print(f"  ├─ Columns: {len(spark_df.columns)}")
    print(f"  └─ Load time: {datetime.now()}")
    
    # Hiển thị 5 dòng đầu
    print(f"\nPreview 5 dòng đầu:")
    spark.sql(f"SELECT * FROM {table_name} LIMIT 5").show(5, truncate=False)
    
    # Hiển thị watermark
    print(f"\nWatermark hiện tại:")
    spark.sql(f"""
        SELECT * FROM water_mark 
        WHERE table_name = '{table_name}'
    """).show(truncate=False)
    
    # Exit với SUCCESS (chỉ 1 lần duy nhất)
    print("\nEXIT: SUCCESS")
    mssparkutils.notebook.exit("SUCCESS")
    
except Exception as e:
    # Exit với FAILED (chỉ 1 lần duy nhất)
    error_msg = f"FAILED: {str(e)}"
    print(f"\nLỖI: {error_msg}")
    import traceback
    traceback.print_exc()
    print(f"\nEXIT: {error_msg}")
    mssparkutils.notebook.exit(error_msg)